In [1]:
import numpy as np
import tensorflow as tf
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
from tensorflow.contrib import slim
from tensorflow.contrib import distributions

from tqdm import tqdm, tqdm_notebook
import os

from sgp_utils import nlog_fitc, nlog_vfe, fitc_pred, vfe_pred

#FLAGS = tf.app.flags.FLAGS
#tf.app.flags.DEFINE_string('img_path','./img',"""Directory for output figs""")
#tf.app.flags.DEFINE_integer('niter',10000,"""Number of iterations""")
#tf.app.flags.DEFINE_integer('burn_in',100,"""Head-start for discriminator""")
#tf.app.flags.DEFINE_integer('print_freq',500,"""How often to display results""")
#tf.app.flags.DEFINE_float('g_lr',2e-5,"""Generator learning rate""")
#tf.app.flags.DEFINE_float('d_lr',1e-4,"""Discriminator learning rate""")
#tf.app.flags.DEFINE_integer('batch_size',500,"""Batch size""")
#tf.app.flags.DEFINE_integer('gen_type',1,"""Generator Type: 0 for Direct; 1 for Gumbel-softmax""")
#tf.app.flags.DEFINE_string('dataset','kin40k',"""Dataset to Use""")
#tf.app.flags.DEFINE_float('val_prc',.2,"""Portion of training data to use for validation""")
#tf.app.flags.DEFINE_integer('n_clusters',100,"""Number of mixture model clusters""")
#tf.app.flags.DEFINE_integer('n_z',500,"""Number of pseudo-inputs""")
#tf.app.flags.DEFINE_integer('num_refs',5,"""Number of (widening) reference distributions""")

## some nonsense to approximate FLAGS in Jupyter

class FLAGS:
    def __getattr__(self, name):
        self.__dict__[name] = FLAGS()
        return self.__dict__[name]

FLAGS = FLAGS()

FLAGS.img_path = './img'
FLAGS.niter = 10000
FLAGS.burn_in = 100
FLAGS.print_freq = 500
FLAGS.g_lr = 2e-5
FLAGS.d_lr = 1e-4
FLAGS.batch_size = 200
FLAGS.gen_type = 1
FLAGS.dataset = 'kin40k'
FLAGS.val_prc = .2
FLAGS.n_clusters = 50
FLAGS.n_z = 100
FLAGS.num_refs = 5
FLAGS.num_sgp_samples = 20
FLAGS.sgp_approx = 'vfe'

In [2]:
def load_kin40k(val_prc=.2):
    
    import pandas as pd

    kin40k_dir = '/Users/mme/Dropbox/_projects/gp_gan/data/kin40k/'

    # Read KIN40K Data
    train_x = pd.read_csv(kin40k_dir+'kin40k_train_data.asc',sep=' ',header=None,skipinitialspace=True).values
    train_y = pd.read_csv(kin40k_dir+'kin40k_train_labels.asc',sep=' ',header=None,skipinitialspace=True).values
    test_x = pd.read_csv(kin40k_dir+'kin40k_test_data.asc',sep=' ',header=None,skipinitialspace=True).values
    test_y = pd.read_csv(kin40k_dir+'kin40k_test_labels.asc',sep=' ',header=None,skipinitialspace=True).values
    train_y = np.squeeze(train_y)
    test_y = np.squeeze(test_y)

    # Normalize KIN40K Data

    x_var = np.var(train_x,axis=0)
    x_mean = np.mean(train_x,axis=0)
    y_var = np.var(train_y)
    y_mean = np.mean(train_y)

    train_x = (train_x-x_mean)/x_var
    test_x = (test_x-x_mean)/x_var
    train_y = (train_y-y_mean)/y_var
    test_y = (test_y-y_mean)/y_var

    # Create a validation set (20% of training data)

    val_indices = np.random.permutation(np.arange(len(train_x)))<int(val_prc*len(train_x))
    val_x = train_x[val_indices,:]
    val_y = train_y[val_indices]
    train_x = train_x[~val_indices,:]
    train_y = train_y[~val_indices]

    return train_x, train_y, val_x, val_y, test_x, test_y

def load_otherdata():
    return None

In [3]:
def kmeans_mixture_model(d,n_clusters=100,random_state=0):
    """
    Cluster x and return cluster centers and cluster widths (variances)
    """
    from sklearn.cluster import MiniBatchKMeans
    km = MiniBatchKMeans(n_clusters=n_clusters,random_state=random_state).fit(d)

    lbls = km.labels_
    cc = km.cluster_centers_

    d_centered = np.array([x - cc[y] for x,y in zip(d,lbls)])
    c_widths = np.array([np.sum(d_centered[lbls==c,:]**2,axis=0)/np.sum(lbls==c) 
        for c in range(n_clusters)])

    weights = np.array([np.sum(lbls==c) for c in range(n_clusters)])
    weights = weights/np.sum(weights)

    mixture_model = {
        'n_components':n_clusters,
        'n_dims':np.shape(d)[1],
        'weights':weights,
        'means':cc,
        'sds':np.sqrt(c_widths)
        }

    return mixture_model

In [4]:
# Gumbel-softmax code taken from: 
# https://github.com/ericjang/gumbel-softmax/blob/master/Categorical%20VAE.ipynb

def sample_gumbel(shape, eps=1e-20): 
    """Sample from Gumbel(0, 1)"""
    U = tf.random_uniform(shape,minval=0,maxval=1)
    return -tf.log(-tf.log(U + eps) + eps)

def gumbel_softmax_sample(logits, temperature): 
    """ Draw a sample from the Gumbel-Softmax distribution"""
    y = logits + sample_gumbel(tf.shape(logits))
    return tf.nn.softmax(y / temperature)

In [5]:
def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

def generator_simple(initial_model,tau_initial=.003,reuse=False):
    """
    Use random noise 'eps' to sample from mixture model
    """
    
    eps_unif = tf.random_uniform((FLAGS.batch_size,
        initial_model['n_components']),dtype=tf.float32)

    eps_gauss = tf.random_normal((FLAGS.batch_size,
        initial_model['n_dims']),dtype=tf.float32)

    with tf.variable_scope("generator", reuse=reuse) as scope:
        
        means = slim.model_variable('means',
                                    shape=np.shape(initial_model['means']),
                                    initializer=tf.constant_initializer(initial_model['means'])
                                   )
        
        sds = slim.model_variable('sds',
                                  shape=np.shape(initial_model['sds']),
                                  initializer=tf.constant_initializer(initial_model['sds'])
                                 )
        
        w = slim.model_variable('weightlogits',
                                shape=(initial_model['n_components']),
                                initializer=tf.zeros_initializer()
                               )
                                      
        tau = slim.model_variable('tau',
                                  shape=np.shape(tau_initial),
                                  initializer=tf.constant_initializer(tau_initial)
                                 )
        
        weights = gumbel_softmax_sample(tf.tile(tf.expand_dims(tf.nn.log_softmax(w),
                                                               axis=0),(FLAGS.batch_size,1)),temperature=tau)
        
        y = tf.reduce_sum(weights[:,:,tf.newaxis] * means[tf.newaxis,:,:], axis=1)
        y += tf.reduce_sum(weights[:,:,tf.newaxis] * sds[tf.newaxis,:,:],axis=1) * eps_gauss
        
        return y, w, tau

def generator_direct(tau=.003,n_layers=3,reuse=False):
    """
    Use random noise 'eps' to sample from mixture model
    """

    eps_unif = tf.random_uniform((FLAGS.batch_size,
        initial_model['n_components']),dtype=tf.float32)

    eps_gauss = tf.random_normal((FLAGS.batch_size,
        initial_model['n_dims'],
        FLAGS.n_z),dtype=tf.float32)

    with tf.variable_scope("generator", reuse=reuse) as scope:
        
        means = slim.model_variable('means',
                                    shape=np.shape(initial_model['means']),
                                    initializer=tf.constant_initializer(initial_model['means'])
                                   )
        
        sds = slim.model_variable('sds',
                                  shape=np.shape(initial_model['sds']),
                                  initializer=tf.constant_initializer(initial_model['sds'])
                                 )
        
        slim.stack(eps_unif, slim.fully_connected, 
                   [100,100,100,initial_model['n_components']], 
                   activation_fn=tf.nn.elu, 
                   scope='fc'
                  )
        
        weights = slim.softmax(eps_unif/tau)
        
        y = tf.reduce_sum(weights[:,:,tf.newaxis] * means[tf.newaxis,:,:], axis=1)
        y += tf.reduce_sum(weights[:,:,tf.newaxis] * sds[tf.newaxis,:,:],axis=1) * eps_gauss
        
        return y, weights

def generator_gumbel(initial_model,tau_initial=1e-5,reuse=False):
    """
    Use random noise 'eps' to sample from gumbel softmax, then sample mixture
    """

    eps_unif = tf.random_uniform((FLAGS.batch_size,
        initial_model['n_components']),dtype=tf.float32)
    
    # switch this to do less?
    #eps_unif = tf.tile(tf.random_uniform(initial_model['n_components'],dtype=tf.float32)[tf.newaxis,:],
    #    batch_size,[batch_size,1])

    eps_gauss = tf.random_normal((FLAGS.batch_size,
        initial_model['n_dims']),dtype=tf.float32)

    with tf.variable_scope("generator", reuse=reuse) as scope:

        #tau = slim.model_variable('tau',
        #    shape=np.shape(tau_initial),
        #    initializer=tf.constant_initializer(tau_initial)
        #    )
        
        #for now, let's try fixed tau
        
        tau = tau_initial
        
        means = slim.model_variable('means',
            shape=np.shape(initial_model['means']),
            initializer=tf.constant_initializer(initial_model['means'])
            )
        
        sds = slim.model_variable('sds',
            shape=np.shape(initial_model['sds']),
            initializer=tf.constant_initializer(initial_model['sds'])
            )
        
        with slim.arg_scope([slim.fully_connected], activation_fn=lrelu):
            net = slim.fully_connected(eps_unif, 256, scope='fc_0')
            
            for i in range(5):
                dnet = slim.fully_connected(net, 256, scope='fc_%d_r0' % (i+1))
                net += slim.fully_connected(dnet, 256, activation_fn=None, scope='fc_%d_r1' % (i+1),
                    weights_initializer=tf.constant_initializer(0.))
                net = lrelu(net)

        T = slim.fully_connected(net, initial_model['n_components'], activation_fn=None, scope='T',
            weights_initializer=tf.constant_initializer(0.))

        #weights = tf.nn.softmax((tf.nn.log_softmax(T,axis=1) + gumbel)/tau,axis=1)

        weights = gumbel_softmax_sample(tf.nn.log_softmax(T,axis=1),temperature=tau)
                
        # dimension should be: (batch_size, n_components, dimension)
        y = tf.reduce_sum(weights[:,:,tf.newaxis] * means[tf.newaxis,:,:], axis=1)
        y += tf.reduce_sum(weights[:,:,tf.newaxis] * sds[tf.newaxis,:,:],axis=1) * eps_gauss
                
        return y, weights

In [6]:
def adversary(y, reuse=False):
    with tf.variable_scope("adversary", reuse=reuse) as scope:
        with slim.arg_scope([slim.fully_connected], activation_fn=lrelu):
            net = slim.fully_connected(y, 256, scope='fc_0')

            for i in range(5):
                dnet = slim.fully_connected(net, 256, scope='fc_%d_r0' % (i+1))
                net += slim.fully_connected(dnet, 256, activation_fn=None, scope='fc_%d_r1' % (i+1),
                                            weights_initializer=tf.constant_initializer(0.))
                net = lrelu(net) 

        T = slim.fully_connected(net, 1, activation_fn=None, scope='T',
                                weights_initializer=tf.constant_initializer(0.))
        T = tf.squeeze(T, [1])
        return T

In [7]:
def reference(d, num_refs=FLAGS.num_refs):
    
    mmref = kmeans_mixture_model(d,n_clusters=1)
    
    ref = distributions.MixtureSameFamily(
        mixture_distribution=distributions.Categorical(
            probs=[.2]*num_refs),
        components_distribution=distributions.MultivariateNormalDiag(
            loc=np.array([mmref['means'][0]]*num_refs).astype(np.float32),
            scale_diag=np.array([mmref['sds'][0]*scale for scale in [2**e for e in range(num_refs)]]).astype(
                np.float32)
        )
    )
    
    return ref.log_prob, ref.sample

#reflogprob, rs = reference(t_x, num_refs = FLAGS.num_refs)

In [8]:
def logdet(matrix):
    chol = tf.cholesky(matrix)
    return 2.0 * tf.reduce_sum(tf.log(tf.real(tf.matrix_diag_part(chol))),reduction_indices=[-1])

def pairwise_distance(x1,x2):
    r1 = tf.reduce_sum(x1**2,axis=2)[:,:,tf.newaxis]
    r2 = tf.reduce_sum(x2**2,axis=2)[:,tf.newaxis,:]
    r12 = tf.matmul(x1,tf.matrix_transpose(x2))
    return r1+r2-2*r12

def nlog_vfe(x,y,m,sls,sfs,noise):

    P = tf.shape(m)[0] # number of pseudo-input samples
    y = tf.tile(tf.expand_dims(tf.cast(y,dtype=tf.float32),0),(P,1))
    x = tf.tile(tf.expand_dims(tf.cast(x,dtype=tf.float32),0),(P,1,1))

    D = tf.shape(x)[2]
    X = tf.shape(x)[1]

    logtp = tf.constant(np.log(2.*np.pi),dtype=tf.float32)

    #m = m + 1e-6*tf.random_normal(tf.shape(m),dtype=tf.float32)

    M = tf.shape(m)[1]

    jitter = 1e-6*tf.eye(M,dtype=tf.float32)[tf.newaxis,:,:]

    xm = pairwise_distance(x,m)
    mm = pairwise_distance(m,m)

    kxm = sfs[:,tf.newaxis,tf.newaxis]*tf.exp(-.5*xm/sls[:,tf.newaxis,tf.newaxis])
    kmm = sfs[:,tf.newaxis,tf.newaxis]*tf.exp(-.5*mm/sls[:,tf.newaxis,tf.newaxis])
    
    kmx = tf.matrix_transpose(kxm)
    kmmi = tf.matrix_inverse(kmm+jitter)

    qmm_diag = tf.reduce_sum(tf.matmul(kxm,kmmi)*kxm,axis=2)
    
    gd = noise[:,tf.newaxis]
    gid = 1/gd

    tr = sfs*tf.cast(X,tf.float32)-tf.reduce_sum(qmm_diag,axis=1)
    
    kmx_gi_kxm = tf.matmul(kmx,kxm)/noise[:,tf.newaxis,tf.newaxis]

    giy = gid*y
    kgiy = tf.reduce_sum(kmx*giy[:,tf.newaxis,:],axis=2)

    inner = kmmi+kmx_gi_kxm+jitter

    covd = logdet(inner)+logdet(kmm+jitter)+tf.log(noise)*tf.cast(X,dtype=tf.float32)
    
    t1 = .5*tf.cast(X,tf.float32)*logtp
    t2 = .5*tf.reduce_sum(y*y*gid,axis=1) - .5*tf.reduce_sum(
        tf.reduce_sum(kgiy[:,:,tf.newaxis]*tf.matrix_inverse(inner),axis=1)*kgiy,axis=1)
    t3 = .5*covd
    t4 = .5*tf.div(tr,noise)
    
    return t1+t2+t3+t4

In [9]:
def random_samples(x,rows_per_sample,n_samples,axis=0):
    indices = tf.random_uniform((n_samples,tf.shape(x)[0]),dtype=tf.float32)
    _, indices = tf.nn.top_k(indices,k=rows_per_sample)
    return tf.gather(x,indices), indices

In [10]:
### MAIN ###

# Load data

if FLAGS.dataset == 'kin40k':
    train_x, train_y, val_x, val_y, test_x, test_y = load_kin40k(val_prc=FLAGS.val_prc)

# PSEUDOCODE HERE FOR THE MOMENT

initial_model = kmeans_mixture_model(train_x,n_clusters=FLAGS.n_clusters)

# specify the generator that will learn this mixture model

if FLAGS.gen_type == 1:
    y, weights, tau = generator_simple(initial_model)
    #y, weights = generator_gumbel(initial_model)
else:
    y, weights = generator_direct(initial_model)

# specify the reference distribution. we need to both evaluate and sample

reflogprob, rs = reference(train_x, num_refs = FLAGS.num_refs)
refsample = tf.cast(rs(sample_shape=FLAGS.batch_size),dtype=tf.float32)

# specify the adversary, which will learn the likelihood ratio
# between generator samples and reference samples

dref = adversary(refsample)
dy = adversary(y,reuse=True)

# temporary assignment of kernel hyperparams using Inverse Gamma
#sls = distributions.InverseGamma(concentration=1., rate=1.).sample(sample_shape=FLAGS.num_sgp_samples)
#sfs = distributions.InverseGamma(concentration=1., rate=1.).sample(sample_shape=FLAGS.num_sgp_samples)
#noise = distributions.InverseGamma(concentration=1., rate=1.).sample(sample_shape=FLAGS.num_sgp_samples)

sls = tf.constant([1.]*FLAGS.num_sgp_samples,dtype=tf.float32)
sfs = tf.constant([1.]*FLAGS.num_sgp_samples,dtype=tf.float32)
noise = tf.constant([.1]*FLAGS.num_sgp_samples,dtype=tf.float32)

# Get SGP stuff
z, z_idx = random_samples(y,FLAGS.n_z,FLAGS.num_sgp_samples) # samples of (full sets of) pseudo-inputs
sgp_nlogprob = nlog_vfe(train_x,train_y,z,sls,sfs,noise)
ref_logprob = tf.reduce_sum(reflogprob(z),axis=1)
dz = tf.reduce_sum(tf.gather(dy,z_idx),axis=1)
    
# prepare for run

dvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "adversary")
gvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "generator")

dloss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=dy, labels=tf.ones_like(dy))
    + tf.nn.sigmoid_cross_entropy_with_logits(logits=dref, labels=tf.zeros_like(dref))
)

gloss = tf.reduce_sum(ref_logprob + sgp_nlogprob + dz, axis=0)

dtrain_step = tf.train.AdamOptimizer(FLAGS.d_lr).minimize(dloss,var_list=dvars)
gtrain_step = tf.train.AdamOptimizer(FLAGS.g_lr).minimize(gloss,var_list=gvars)

outdir = FLAGS.img_path
if not os.path.exists(outdir):
    os.makedirs(outdir)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updatin

In [11]:
def run_training(sess, niter=FLAGS.niter, ninitial=FLAGS.burn_in, figdir=FLAGS.img_path):

    # consider a burn-in period for the discriminator before going to the generator

    burn_in = tqdm_notebook(range(ninitial))

    for i in burn_in:

        y_,weights_,tau_,refsample_,dref_,dy_,dloss_,_ = sess.run([y,weights,tau,refsample,dref,dy,dloss,dtrain_step])

        burn_in.set_description("dloss=%.3f"  % (dloss_))

    progress = tqdm_notebook(range(niter))

    for i in progress:

        y_,weights_,refsample_,dref_,dy_,dloss_,_ = sess.run([y,weights,refsample,dref,dy,dloss,dtrain_step])
        gloss_, _ = sess.run([gloss,gtrain_step])

        #print(np.shape(y_),np.shape(refsample_))

        progress.set_description("dloss=%.3f, gloss=%.3f"  % (dloss_,gloss_))
        
        if i%10 == 0:
            #unique, counts = np.unique(weights_, return_counts=True)
            #print(np.array((unique, counts)).T)
            #print(tau_)

In [12]:
try:
    s.close()
except NameError:
    pass
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())
run_training(s,niter=FLAGS.niter)

[[ 0. 50.]]
0.003
[[-1.04187100e-04  1.00000000e+00]
 [-9.16260178e-05  1.00000000e+00]
 [-8.66371993e-05  1.00000000e+00]
 [-8.37518091e-05  1.00000000e+00]
 [-8.07654142e-05  1.00000000e+00]
 [-7.65696968e-05  1.00000000e+00]
 [-7.56021764e-05  1.00000000e+00]
 [-7.44973586e-05  1.00000000e+00]
 [-7.43167184e-05  1.00000000e+00]
 [-7.36162110e-05  1.00000000e+00]
 [-7.33930938e-05  1.00000000e+00]
 [-7.21911128e-05  1.00000000e+00]
 [-7.20259341e-05  1.00000000e+00]
 [-6.77150601e-05  1.00000000e+00]
 [-5.46998999e-05  1.00000000e+00]
 [-5.40253823e-05  1.00000000e+00]
 [-5.39434332e-05  1.00000000e+00]
 [-5.10986465e-05  1.00000000e+00]
 [-5.05556745e-05  1.00000000e+00]
 [-4.83980439e-05  1.00000000e+00]
 [-4.18237287e-05  1.00000000e+00]
 [-3.12371667e-05  1.00000000e+00]
 [-2.81949542e-05  1.00000000e+00]
 [-2.80933600e-05  1.00000000e+00]
 [-2.36414180e-05  1.00000000e+00]
 [-8.31566285e-06  1.00000000e+00]
 [-2.84359521e-06  1.00000000e+00]
 [ 1.02606054e-06  1.00000000e+00]
 [

[[-3.51448689e-04  1.00000000e+00]
 [-2.95320933e-04  1.00000000e+00]
 [-2.88165116e-04  1.00000000e+00]
 [-2.87940464e-04  1.00000000e+00]
 [-2.78592226e-04  1.00000000e+00]
 [-2.63042137e-04  1.00000000e+00]
 [-2.22077360e-04  1.00000000e+00]
 [-2.15295411e-04  1.00000000e+00]
 [-2.08824393e-04  1.00000000e+00]
 [-1.78791277e-04  1.00000000e+00]
 [-1.76063593e-04  1.00000000e+00]
 [-1.42353223e-04  1.00000000e+00]
 [-1.16562362e-04  1.00000000e+00]
 [-1.08531654e-04  1.00000000e+00]
 [-1.02810780e-04  1.00000000e+00]
 [-1.02470403e-04  1.00000000e+00]
 [-8.35078972e-05  1.00000000e+00]
 [-7.78491521e-05  1.00000000e+00]
 [-6.79749865e-05  1.00000000e+00]
 [-5.68749965e-05  1.00000000e+00]
 [-5.27614975e-05  1.00000000e+00]
 [-4.06089457e-05  1.00000000e+00]
 [-3.34898104e-05  1.00000000e+00]
 [-1.72346736e-05  1.00000000e+00]
 [-1.05337740e-05  1.00000000e+00]
 [-4.95519771e-06  1.00000000e+00]
 [-1.03155685e-06  1.00000000e+00]
 [ 4.52092308e-06  1.00000000e+00]
 [ 9.69451048e-06  1

KeyboardInterrupt: 

In [ ]:
# find more realistic values of kernel hyperparams
# use ARD kernel
# figure out how to train kernel
# use datasets from titsias
# experiment with number of pips
# display distribution over clusters

# From Ricardo:
# smaller number of pseudo-inputs -- say 100-200
# this number should be similar to the number of clusters
# make sure pseudo-inputs aren't too similar
# this could prevent kernel from collapsing

# the ability to check pseudo-inputs for similarity: can't do it
# hey, i'm going to select pseudo-inputs, and when I draw I only get 150 that are different, I only draw 150
# in contrast, nothing you can do about this with previous way

